## Homework: Deep Jazz

In [1]:
import numpy as np
from music21 import stream, midi, tempo, note

from grammar import unparse_grammar
from preprocess import get_musical_data, get_corpus_data

from qa import prune_grammar, prune_notes, clean_up_notes
from generator import __sample, __generate_grammar, __predict

In [2]:
N_epochs = 128  # default
data_fn = 'midi/' + 'original_metheny.mid'  # 'And Then I Knew' by Pat Metheny
out_fn = 'midi/' 'deepjazz_on_metheny...' + str(N_epochs)

In [3]:
max_len = 20
max_tries = 1000
diversity = 0.5

# musical settings
bpm = 130

# get data
chords, abstract_grammars = get_musical_data(data_fn)
corpus, values, val_indices, indices_val = get_corpus_data(abstract_grammars)
print('corpus length:', len(corpus))
print('total # of values:', len(values))

corpus length: 193
total # of values: 78


In [4]:
chords[0]

[<music21.instrument.Piano Piano>,
 <music21.tempo.MetronomeMark Quarter=112.0>,
 <music21.key.Key of G major>,
 <music21.meter.TimeSignature 4/4>]

In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

def get_keras_model(max_len, N_values):
    # build a 2 stacked LSTM
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(max_len, N_values)))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(N_values))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

ModuleNotFoundError: No module named 'keras'

## Task

Replace previous model with equivalent in prettytensor or tf.slim

Try to make you code as compact as possible

In [5]:
import prettytensor as pt
import tensorflow as tf
from tensorflow.contrib import rnn

class DeepJazzModel:
    def __init__(self, max_len, N_values):
        self.max_len = max_len
        self.N_values = N_values
        self.x = tf.placeholder(tf.float32, shape=[None, max_len, N_values], name='x')
        self.y = tf.placeholder(tf.float32, [None, self.N_values], name = 'y')
        self.sess = tf.Session()
    
    def __create_network(self, training=True):
        lstm_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(128, forget_bias=1.0), rnn.BasicLSTMCell(128, forget_bias=1.0)])
        outputs, states = tf.nn.dynamic_rnn(lstm_cell, self.x, dtype=tf.float32)
        last_out = tf.transpose(outputs, [1, 0, 2])[-1]
        
        weights = tf.Variable(tf.truncated_normal([128, self.N_values], stddev=0.05), trainable=True)
        baises = tf.Variable(tf.truncated_normal(shape=[self.N_values], stddev=0.05))
        fc = tf.matmul(last_out, weights) + baises
        
        self.y_pred = tf.nn.softmax(fc)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=fc, labels=self.y)
        self.loss = tf.reduce_mean(cross_entropy)
    
    
    def random_batch(self, X, y, batch_size):
        num = len(X)
        idx = np.random.choice(num,
                           size=batch_size,
                           replace=False)
        x_batch = X[idx, :, :]
        y_batch = y[idx, :]

        return x_batch, y_batch
    
    def fit(self, X, y, batch_size, epochs):
        global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
        self.__create_network()
        optimizer = tf.train.RMSPropOptimizer(learning_rate=1e-4).minimize(self.loss, global_step=global_step)
        self.sess.run(tf.global_variables_initializer())
        
        for i in range(epochs):
            x_batch, y_true_batch = self.random_batch(X, y, batch_size)
            feed_dict_train = {self.x: x_batch, self.y: y_true_batch}
            self.sess.run(optimizer, feed_dict=feed_dict_train)
    
    
    #model.predict(x, verbose=0)[0]
    def predict(self, x, verbose):
        feed_dict = {self.x: x}
        #y_pred_class = tf.argmax(self.y_pred, axis=1)
        y_pred = self.sess.run(self.y_pred, feed_dict=feed_dict)
        return y_pred
    

In [6]:
def get_pretty_tensor_model(max_len, N_values):
    return DeepJazzModel(max_len, N_values)

In [7]:
get_model = get_pretty_tensor_model

In [8]:
import numpy as np

''' Build a 2-layer LSTM from a training corpus '''


def build_model(corpus, val_indices, max_len, N_epochs=128):
    # number of different values or words in corpus
    N_values = len(set(corpus))

    # cut the corpus into semi-redundant sequences of max_len values
    step = 3
    sentences = []
    next_values = []
    for i in range(0, len(corpus) - max_len, step):
        sentences.append(corpus[i: i + max_len])
        next_values.append(corpus[i + max_len])
    print('nb sequences:', len(sentences))

    # transform data into binary matrices
    X = np.zeros((len(sentences), max_len, N_values), dtype=np.bool)
    y = np.zeros((len(sentences), N_values), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, val in enumerate(sentence):
            X[i, t, val_indices[val]] = 1
        y[i, val_indices[next_values[i]]] = 1

    model = get_model(max_len, N_values)
    model.fit(X, y, batch_size=10, epochs=N_epochs)

    return model


In [9]:
import tensorflow as tf

# build model
model = build_model(corpus=corpus, val_indices=val_indices,
                         max_len=max_len, N_epochs=N_epochs)

# set up audio stream
out_stream = stream.Stream()

# generation loop
curr_offset = 0.0
loopEnd = len(chords)
for loopIndex in range(1, loopEnd):
    # get chords from file
    curr_chords = stream.Voice()
    for j in chords[loopIndex]:
        curr_chords.insert((j.offset % 4), j)

    # generate grammar
    curr_grammar = __generate_grammar(model=model, corpus=corpus,
                                      abstract_grammars=abstract_grammars,
                                      values=values, val_indices=val_indices,
                                      indices_val=indices_val,
                                      max_len=max_len, max_tries=max_tries,
                                      diversity=diversity)

    curr_grammar = curr_grammar.replace(' A', ' C').replace(' X', ' C')

    # Pruning #1: smoothing measure
    curr_grammar = prune_grammar(curr_grammar)

    # Get notes from grammar and chords
    curr_notes = unparse_grammar(curr_grammar, curr_chords)

    # Pruning #2: removing repeated and too close together notes
    curr_notes = prune_notes(curr_notes)

    # quality assurance: clean up notes
    curr_notes = clean_up_notes(curr_notes)

    # print # of notes in curr_notes
    print('After pruning: %s notes' % (len([i for i in curr_notes
                                            if isinstance(i, note.Note)])))

    # insert into the output stream
    for m in curr_notes:
        out_stream.insert(curr_offset + m.offset, m)
    for mc in curr_chords:
        out_stream.insert(curr_offset + mc.offset, mc)

    curr_offset += 4.0

out_stream.insert(0.0, tempo.MetronomeMark(number=bpm))

# Play the final stream through output (see 'play' lambda function above)
play = lambda x: midi.realtime.StreamPlayer(x).play()
play(out_stream)

# save stream
mf = midi.translate.streamToMidiFile(out_stream)
mf.open(out_fn, 'wb')
mf.write()
mf.close()    

nb sequences: 58
After pruning: 13 notes
After pruning: 12 notes
After pruning: 11 notes
After pruning: 14 notes
After pruning: 13 notes
After pruning: 15 notes
After pruning: 14 notes
After pruning: 13 notes
After pruning: 12 notes
After pruning: 12 notes
After pruning: 13 notes
After pruning: 17 notes
After pruning: 14 notes
After pruning: 13 notes
After pruning: 12 notes
After pruning: 14 notes
After pruning: 13 notes
After pruning: 11 notes


You can play generated sample using any midi player

Under linux I prefer timidity

In [13]:
!! timidity midi/deepjazz_on_metheny...128

['Requested buffer size 32768, fragment size 8192',
 "ALSA pcm 'default' set buffer size 32768, period size 8192 bytes",
 'Playing midi/deepjazz_on_metheny...128',
 'MIDI file: midi/deepjazz_on_metheny...128',
 'Format: 1  Tracks: 1  Divisions: 1024',
 'Sequence: ',
 'Playing time: ~38 seconds',
 'Notes cut: 0',
 'Notes lost totally: 0']